# RAMEN RATINGS

We had acess to the ratings of a prestigious site of instant ramen with the following informations: <br /> 

**Review #**: the order of the rating <br /> 
**Brand**: the name of the brand <br /> 
**Variety**: the nam of the product <br /> 
**Style**: the format of the ramen package <br /> 
**Country**: brand's country<br /> 
**Stars**: the rating <br /> 
**Top Ten**: the top 10 rating per year <br /> 

Link of the dataset on Kaggle:
https://www.kaggle.com/residentmario/ramen-ratings

We aim to answer the following questions:
*     How is ramen manufacturing internationally distributed?
*     How are the ramen ratings distributed per the greater ramen productors? 
*     Is there a relation between the ratings and the product's style? 


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# For importing
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# For pre-processing
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# For data visualization
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import geopandas as gpd

In [2]:
ramen_list = pd.read_csv('/kaggle/input/ramen-ratings/ramen-ratings.csv')
ramen_list.head(5)

# **Data manipulation**

## 1. Evaluating categorical values written twice differently:

In [23]:
brands= ramen_list["Brand"].unique()
brands.sort()
brands

We see that there are some brands with duplicated nouns with some variation as 'Wu Mu', 'Chorip Dong' and 'Fashion Food' and 'Mama'.
So we're going to filter some of this cases with the follwing function:

In [4]:
# Replaces rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

In [5]:
replace_matches_in_column(df=ramen_list, column="Brand", string_to_match="Chorip Dong", min_ratio = 47)
replace_matches_in_column(df=ramen_list, column="Brand", string_to_match="Fashion Food", min_ratio = 47)
replace_matches_in_column(df=ramen_list, column="Brand", string_to_match="Mama", min_ratio = 47)
replace_matches_in_column(df=ramen_list, column="Brand", string_to_match="Wu Mu", min_ratio = 47)

In [6]:
ramen_list["Style"].unique()

In [7]:
countries= ramen_list["Country"].unique()
countries.sort()
countries

In [8]:
duplicates = ramen_list["Review #"].duplicated().sum()
duplicates

## 2.  Evaluating missing row values per columns:

In [9]:
missing_values_counter = ramen_list.isnull().sum()
print(missing_values_counter[0:7])

lines_quantity = len(ramen_list)
print("Quantity of rows/lines:",lines_quantity)

In [10]:
ramen_list_fill_style = ramen_list.copy()

# Locate the null cells
print("Null cells: \n",ramen_list_fill_style.loc[(ramen_list_fill_style['Style'].isnull())],"\n",80*"_")

In [11]:
# Assigning neighbor values in the column 'Style' for missing cells
ramen_list_fill_style['Style'] = ramen_list_fill_style['Style'].fillna(method='bfill', axis=0).fillna(0)

# To confirm if it worked
ramen_list_fill_style.loc[(ramen_list_fill_style['Style'].isnull())]

In [12]:
# Evaluating the stars character type
type(ramen_list_fill_style["Stars"][1])

In [13]:
# Converting the column's format to numerical and verifying if there's no null value
ramen_list_fill_style["Stars"] = pd.to_numeric(ramen_list_fill_style["Stars"], errors='coerce')
#Verifying if there is null values
ramen_list_fill_style.loc[ramen_list_fill_style["Stars"].isnull()]

In [22]:
# Adopting the same value from the cell above in the column 'Stars' to the null values evaluations
ramen_list_fill_style['Stars'] = ramen_list_fill_style['Stars'].fillna(method='bfill', axis=0).fillna(0)

# To confirm if it worked
ramen_list_fill_style.loc[(ramen_list_fill_style['Stars'].isnull())]

## 3. Data Visualization

In [15]:
plt.figure(figsize=(12,10))
sns.countplot(data=ramen_list_fill_style, y="Country", order=ramen_list_fill_style['Country'].value_counts().index)
plt.title('Countries X Ramen brand quantity')
plt.xlabel('Number of ramen brand')
plt.ylabel("Countries")
#plt.legend(loc='best') # loc=best - legend of the dataset 
plt.show()

In [16]:
# Style percentage
style_per_quantity = ramen_list_fill_style["Style"].value_counts()/(len(ramen_list_fill_style))*100
print("Style percentage:\n",style_per_quantity)

In [17]:
ramen_list_fill_style.describe()

From the above metrics, we can see that a significantly part of the ratings are above 2.5. More specifically, 75% have a rating greater than 3.25.

In [18]:
plt.figure(figsize=(13,5))
plt.hist(ramen_list_fill_style['Stars'],bins=15, histtype='bar')
plt.title('Histogram: Ramen''s evaluations')
plt.xlabel('Grade attributed to the ramens')
plt.ylabel("Frequence of the evaluations")
plt.show()

## Crossing dimensions

In [19]:
# Box plotting ramen rating x style of the package
plt.figure(figsize=(13,5))
sns.boxplot(x=ramen_list_fill_style["Style"],y=ramen_list_fill_style["Stars"], order=['Pack','Bowl','Cup','Tray','Box','Can','Bar'])
sns.stripplot(x=ramen_list_fill_style["Style"], y=ramen_list_fill_style["Stars"],order=['Pack','Bowl','Cup','Tray','Box','Can','Bar'])
plt.xlabel('Package style')
plt.ylabel('Ramen evaluation')
plt.show()

Selecting the 5 countries that owns more brands in order to do map their ratings: 

In [20]:
# Box plotting ramen rating x style of the package
plt.figure(figsize=(13,5))
sns.boxplot(x=ramen_list_fill_style["Country"],y=ramen_list_fill_style["Stars"], order=['Japan','Taiwan','USA','South Korea','Thailand'])
plt.xlabel('Country')
plt.ylabel('Ramen evaluation')
plt.show()

## 4. Conclusions

Summaring some of our observations. We have:
*     How is ramen manufacturing internationally distributed?</br>
The major productors countries are at the Asian continent. With Japan as the owner of more than 320 brands.
*     How are the ramen ratings distributed per the greater ramen productors?</br>
Japan is the country with more than 50% of its ramen production score above 4.0.
*     Is there a relation between the ratings and the product's style?</br>
The 'pack' format is responsible for about 59.4% of the evaluations. Considering the 3 major products formats(pack, bowl and cup), the bowl type received a better evaluation for more than 50% of its products. 